## Paralelización de la preparaición de datos para un modelo profundo

En este programa se va implementar la preparación paralela de datos para dar como entrada a un algoritmo de aprendizaje profundo, en este caso se realizará un cambio de tamaño en la imagen

In [1]:
from multiprocessing import Value, Lock, Array, Lock, Process, cpu_count
import os
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
# Imprimir cuántos cores tiene la computadora
cores = cpu_count()
print('Cores en la computadora:', cores)

Cores en la computadora: 4


In [3]:

def load_data(dataset):
    raw=[]
    Y=[]
    y=0
    for folder in [x[0] for x in os.walk(dataset)][1:]:
        for image_name in next(os.walk(folder), (None, None, []))[2][:100]:
            raw.append(cv2.imread(folder+'/'+image_name))
            Y.append(y)
        y+=1
    return raw, Y

In [4]:
X, y = load_data('/media/jesus/DATA/Datasets/COVID/Kaggle/')

In [5]:
def resize_images(contador, lock):
    while(len(X)>contador.value):
        with contador.get_lock():
            elemento=contador.value
            contador.value+=1
        #print(X[elemento].shape)
        cv2.resize(X[elemento], (224,224), interpolation = cv2.INTER_AREA)
        

In [6]:
times=[]
for i in range(cores):
    threads=[]
    N_THREADS=i+1

    contador=Value('i',0)
    lock = Lock()

    for i in range(N_THREADS):
        threads.append(Process(target=resize_images, args=[contador, lock]))


    start_time = time.perf_counter()
    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

    finish_time = time.perf_counter()
    print(f"Program finished in {finish_time-start_time} seconds")
    times.append(finish_time-start_time)

Program finished in 1.928216042000713 seconds
Program finished in 1.3309367149995524 seconds
Program finished in 1.1049006730008841 seconds
Program finished in 0.9783716380006808 seconds
